# Steps from Egan's paper

1. Preprocessing
The paper also proposes preprocessing steps, however I follow the preprocessing steps that we already did.

2. Epoching
Create trial-length epochs for alpha, these span the entire trial (20 seconds), they start 0.5 seconds before the trial until 0.5 seconds after the trial. For the P300 aalysis, create target-locked epochs, starting 0.25 seconds before the target and ending 1 second after the target.

3. Frequency Analysis for Alpha Band
Perform frequency analysis using a 2-second rectangular sliding window across each trial-length epoch in steps of 0.5 seconds. For each window, apply a FFT to the data from each electrode. Extract the average amplitude between 8–13 Hz (alpha band) for each step. Average the features within a selected set of electrodes to obtain the  alpha values that will be used for classification.

4. ERP Feature Extraction for P300
Separate target-locked epochs for attended and unattended stimuli. Average the target-locked epochs across occurrences of attended and unattended targets. Baseline correct the ERP by subtracting the average of the 0.5 seconds preceding the appearance of the target from the remaining data (0–0.75 s).

5. Combine the frequency domain features from the alpha-band analysis with the ERP features from the P300 analysis.
Ensure that the features are concatenated in a way that is suitable for classification, typically forming a combined feature vector for each trial.

6. Classification
Train an LDA classifier using the hybrid feature set (alpha-band + P300 ERP features), possibly with ledoit wolf regularization. Use cross-validation to evaluate classification performance and avoid overfitting.

7. Evaluation
Evaluate the classification performance using appropriate metrics such as accuracy, precision, recall, F1 score, or ROC curves.

### Questions:
Should I use binning of the ERP epochs?

# Approach 1: Most similar to Egan's paper:

c-VEP --> per electrode (scalp map links en rechts) correlatie met die trial en template links. Voor links heb je dan Rho links en rechts Rho rechts. Geen CCA, handmatig kiezen van spatieel filter. mV^2 mV en correlatie.
P300 pynthbci to epoch

In [172]:
import numpy as np
import os
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from mne.time_frequency import psd_array_welch
import mne

# Suppress warnings from MNE
mne.set_log_level('warning')

# Define path
file_dir = '/Users/juliette/Desktop/thesis/preprocessing/hybrid_preprocessing'
decoding_results_dir = '/Users/juliette/Desktop/thesis/results/alpha+p300'

# Set parameters
window_size = 2     # in seconds for alpha-band sliding window
step_size = 0.5     # sliding step
min_bin, max_bin = 8, 13  # alpha band

# Subjects
subjects = ["VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij",
            "VPpdik", "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit",
            "VPpdiu", "VPpdiv", "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"]

# Initialize results storage
results = []

# Loop over the subjects
for subject in subjects:
    print(f"Processing {subject}...")

    # Load preprocessed ICA-cleaned data
    fn = os.path.join(file_dir, f"sub-{subject}_task-covert_c-VEP+P300_ICA.npz")
    tmp = np.load(fn, allow_pickle=True)

    X = tmp["X"]  # shape: trials × channels × samples
    y = tmp["y"]  # binary trial label (e.g., attention)
    z = tmp["z"]  # stimulus labels
    V = tmp["V"]  # marker stream, used for timing info
    fs = int(tmp["fs"].flatten()[0])
    n_trials, n_channels, n_samples = X.shape

    # Split folds
    fold_accuracies = []
    n_folds = 4
    n_per_fold = n_trials // n_folds
    folds = np.repeat(np.arange(n_folds), n_per_fold)

    for i_fold in range(n_folds):
        print(f"  Fold {i_fold + 1}/{n_folds}")

        # Train/test split
        X_trn, y_trn, z_trn = X[folds != i_fold,:,:], y[folds != i_fold], z[folds != i_fold]
        X_tst, y_tst, z_tst = X[folds == i_fold,:,:], y[folds == i_fold], z[folds == i_fold]

        # Frequency analysis for the alpha band
        def extract_alpha(X_trials):
            alpha_feats = []
            for trial in X_trials:
                trial_alpha = [] # This variable will hold the alpha power values for each time window in the current trial
                total_duration = n_samples/fs
        
                for start in np.arange(0, total_duration - window_size, step_size):
                    start_ix = int(start * fs) # convert the starting times to sample indices
                    stop_ix = int(start_ix + window_size * fs)
                    
                    # If stop_ix exceeds the time samples in the trial the window is too large and it stops
                    if stop_ix > trial.shape[1]:
                        break
                    
                    # This contains the EEG data for the selected time window
                    # It is sliced from the trial array for all channels, and the selected time window
                    segment = trial[:, start_ix:stop_ix]
                    
                    psds, freqs = psd_array_welch(
                        segment, sfreq=fs, fmin=min_bin, fmax=max_bin, n_per_seg=segment.shape[1] # The number of points per segment 
                    )                                                                             # is equal to the length of the segment
                    alpha_power = psds.mean(axis=1)  # average power across the frequency bins in the alpha band across channels
                    trial_alpha.append(alpha_power)
                trial_alpha = np.mean(trial_alpha, axis=0)  # calculate single alpha power feature for the trial
                alpha_feats.append(trial_alpha)
            return np.array(alpha_feats)  # shape: (trials × channels)

        alpha_trn = extract_alpha(X_trn)
        alpha_tst = extract_alpha(X_tst)

        # --- STEP 4: ERP FEATURE EXTRACTION FOR P300 ---
        def extract_erp_no_bins(X_trials, z_trials, y, fs, n_channels, n_bins=6):
            # An empty list that will hold the ERP features for each trial
            erp_feats = []

            # Extracting all the stimulus coding
            side_tst = z_trials[:, :, :]  # All epochs for each trial (n_trials, n_epochs, 2)

            # Identify left and right target events
            z_left = side_tst[:, :, 0] == 1  # Left target (index 0)
            z_right = side_tst[:, :, 1] == 1  # Right target (index 1)

            # Loop through each trial, stimulus coding and attended side label
            for trial, z, target_side in zip(X_trials, z_trials, y):
                # Determine attended and unattended sides
                if target_side == 0:  # Attending to left
                    attended_side = z_left  # Left target is attended
                    unattended_side = z_right  # Right target is unattended
                else:  # Attending to right
                    attended_side = z_right  # Right target is attended
                    unattended_side = z_left  # Left target is unattended

                # Find the indices of targets on the attended side
                target_indices_attended = np.where(attended_side == 1)[0]
                # Find the indices of targets on the unattended side
                target_indices_unattended = np.where(unattended_side == 1)[0]

                # If no target indices are found for both the attended and unattended sides, append a placeholder
                if len(target_indices_attended) == 0 and len(target_indices_unattended) == 0:
                    erp_feats.append(np.zeros(n_channels * 2))  # Placeholder (attended + unattended)
                    continue

                # Initialize arrays that will hold the ERP features for the attended and unattended targets
                feat_attended = np.zeros(n_channels)
                feat_unattended = np.zeros(n_channels)

                # Process attended target
                if len(target_indices_attended) > 0:
                    # Extract the first target index, which gives the position of the attended target
                    t_ix_attended = target_indices_attended[0]
                    start_ix_attended = int(t_ix_attended - 0.25 * fs)  # 250 ms before the target (baseline period)
                    stop_ix_attended = int(t_ix_attended + 1.0 * fs)    # 1000 ms after the target

                    # Ensure indices are within the valid range for the current trial
                    start_ix_attended = max(0, start_ix_attended)
                    stop_ix_attended = min(trial.shape[1], stop_ix_attended)

                    # Extract the EEG data around the attended target using the indices
                    epoch_attended = trial[:, start_ix_attended:stop_ix_attended]

                    # Compute the ERP feature for the attended target by averaging the EEG data across the time axis
                    feat_attended = epoch_attended.mean(axis=1)

                # Process unattended target
                if len(target_indices_unattended) > 0:
                    # Extract the first target index, which gives the position of the unattended target
                    t_ix_unattended = target_indices_unattended[0]
                    start_ix_unattended = int((t_ix_unattended - 0.25) * fs)  # 250 ms before the target (baseline period)
                    stop_ix_unattended = int(t_ix_unattended + 1.0 * fs)    # 1000 ms after the target

                    # Ensure indices are within the valid range for the current trial
                    start_ix_unattended = max(0, start_ix_unattended)
                    stop_ix_unattended = min(trial.shape[1], stop_ix_unattended)

                    # Extract the EEG data around the unattended target using the indices
                    epoch_unattended = trial[:, start_ix_unattended:stop_ix_unattended]

                    # Compute the ERP feature for the unattended target by averaging the EEG data across the time axis
                    feat_unattended = epoch_unattended.mean(axis=1)

                # Concatenate attended and unattended features and append to the list
                erp_feats.append(np.concatenate([feat_attended, feat_unattended]))

            return np.array(erp_feats)


        # Extract ERP features for training and testing
        erp_trn = extract_erp_no_bins(X_trn, z_trn, y_trn, fs, n_channels)
        erp_tst = extract_erp_no_bins(X_tst, z_tst, y_tst, fs, n_channels)
        print("ERP_trn:", erp_trn)

        # --- STEP 5: FEATURE FUSION ---
        F_trn = np.concatenate([alpha_trn, erp_trn], axis=1)
        F_tst = np.concatenate([alpha_tst, erp_tst], axis=1)

        # Handle NaNs, replaces any NaN values with 0
        F_trn[np.isnan(F_trn)] = 0
        F_tst[np.isnan(F_tst)] = 0

        # Normalize, it scales the features so that they have a mean of 0 and a standard deviation of 1
        scaler = StandardScaler()
        F_trn = scaler.fit_transform(F_trn)
        F_tst = scaler.transform(F_tst)

        # --- STEP 6: CLASSIFICATION ---
        clf = LDA(solver='lsqr', shrinkage='auto')  # Ledoit-Wolf shrinkage
        clf.fit(F_trn, y_trn)
        y_pred = clf.predict(F_tst)

        acc = np.mean(y_pred == y_tst)
        fold_accuracies.append(acc)
        print(f"    Fold accuracy: {acc:.2f}")

    # --- STEP 7: SAVE RESULTS FOR THIS SUBJECT ---
    acc_mean = np.round(np.mean(fold_accuracies), 2)
    acc_se = np.round(np.std(fold_accuracies) / np.sqrt(n_folds), 2)
    results.append((subject, acc_mean, acc_se))
    print(f"{subject}: Accuracy = {acc_mean:.2f}, SE = {acc_se:.2f}")

    
# --- STEP 8: AVERAGE ACCURACY OVER ALL SUBJECTS ---
all_accuracies = [result[1] for result in results]
mean_accuracy_all = np.mean(all_accuracies)
se_accuracy_all = np.std(all_accuracies) / np.sqrt(len(all_accuracies))

print(f"\nAverage Accuracy across all subjects: {mean_accuracy_all:.2f}")
print(f"Standard Error of Accuracy: {se_accuracy_all:.2f}")


Processing VPpdia...
  Fold 1/4
ERP_trn: [[[-2.20337902e-07 -6.45332767e-08  4.85096478e-08 ... -3.18792233e-07
   -2.63607704e-07 -4.45821188e-08]
  [-2.20337902e-07 -6.45332767e-08  4.85096478e-08 ... -3.18792233e-07
   -2.63607704e-07 -4.45821188e-08]]

 [[ 3.36258625e-07  1.78890340e-07  1.33797484e-07 ... -2.42158882e-07
   -2.82308764e-07 -7.80666164e-08]
  [ 3.36258625e-07  1.78890340e-07  1.33797484e-07 ... -2.42158882e-07
   -2.82308764e-07 -7.80666164e-08]]

 [[-8.17316834e-08 -4.82179499e-08  1.51316219e-07 ...  1.25846834e-07
    6.04532904e-08  2.42081740e-08]
  [-8.17316834e-08 -4.82179499e-08  1.51316219e-07 ...  1.25846834e-07
    6.04532904e-08  2.42081740e-08]]

 ...

 [[ 2.74897451e-07  4.41581187e-07  2.48312974e-07 ... -6.97453142e-07
   -7.16567370e-07 -3.22376195e-07]
  [ 2.74897451e-07  4.41581187e-07  2.48312974e-07 ... -6.97453142e-07
   -7.16567370e-07 -3.22376195e-07]]

 [[-6.29110097e-08 -1.98116739e-07 -2.03052887e-07 ... -2.66552698e-07
    5.50207625e-08

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [170]:
def extract_erp_no_bins(X_trials, z_trials, y, fs, n_channels, amplitude_threshold=40e-6, start_sample_offset=-24, end_sample_offset=84, step_size=30):
    # An empty list that will hold the ERP features for each trial
    erp_feats = []

    # Loop through each trial, stimulus coding, and attended side label
    for trial, z, target_side in zip(X_trials, z_trials, y):
        cued_side = target_side  # Assuming 'y' contains the trial information about which side (left or right)

        # Create event vectors for left and right targets
        left_targets = z[:, 0]  # Assuming z contains the event markers, where 0 = left, 1 = right
        right_targets = z[:, 1]
        attended_stimuli = z[:, cued_side]  # Select the cued target side

        # Identify left and right target events
        z_left = left_targets == 1
        z_right = right_targets == 1

        # Determine attended and unattended sides
        if target_side == 0:  # Attending to left
            attended_side = z_left  # Left target is attended
            unattended_side = z_right  # Right target is unattended
        else:  # Attending to right
            attended_side = z_right  # Right target is attended
            unattended_side = z_left  # Left target is unattended

        # Find the indices of targets on the attended side
        target_indices_attended = np.where(attended_side == 1)[0]
        # Find the indices of targets on the unattended side
        target_indices_unattended = np.where(unattended_side == 1)[0]

        # If no target indices are found for both the attended and unattended sides, append a placeholder
        if len(target_indices_attended) == 0 and len(target_indices_unattended) == 0:
            erp_feats.append(np.zeros(n_channels * 2))  # Placeholder (attended + unattended)
            continue

        # Initialize arrays that will hold the ERP features for the attended and unattended targets
        feat_attended = np.zeros(n_channels)
        feat_unattended = np.zeros(n_channels)

        # Process attended target
        if len(target_indices_attended) > 0:
            # Extract the first target index, which gives the position of the attended target
            t_ix_attended = target_indices_attended[0]
            print("t_ix_attended:", t_ix_attended)
            
            # Extract epochs around the attended target
            epoch_attended, _ = extract_epochs(trial[None, :, :], start_idx=120, end_idx=2520, step_size=30, 
                   start_sample_offset=-24, end_sample_offset=84, 
                   amplitude_threshold=40e-6)
            
            # Compute the ERP feature for the attended target by averaging the EEG data across the time axis
            feat_attended = epoch_attended.mean(axis=(1, 2))  # Averaging across epochs and channels

        # Process unattended target
        if len(target_indices_unattended) > 0:
            # Extract the first target index, which gives the position of the unattended target
            t_ix_unattended = target_indices_unattended[0]
            print("t_ix_unattended:", t_ix_unattended)
            
            # Extract epochs around the unattended target
            epoch_unattended, _ = extract_epochs(trial[None, :, :], start_idx=120, end_idx=2520, step_size=30, 
                   start_sample_offset=-24, end_sample_offset=84, 
                   amplitude_threshold=40e-6)
            
            # Compute the ERP feature for the unattended target by averaging the EEG data across the time axis
            feat_unattended = epoch_unattended.mean(axis=(1, 2))  # Averaging across epochs and channels

        # Concatenate attended and unattended features and append to the list
        erp_feats.append(np.concatenate([feat_attended, feat_unattended]))

    return np.array(erp_feats)

# Example usage
erp_trn = extract_erp_no_bins(X_trn, z_trn, y_trn, fs, n_channels)
erp_tst = extract_erp_no_bins(X_tst, z_tst, y_tst, fs, n_channels)

# Print the result
print("ERP_trn:", erp_trn)


t_ix_attended: 11
t_ix_unattended: 4
t_ix_attended: 4
t_ix_unattended: 7
t_ix_attended: 7
t_ix_unattended: 12
t_ix_attended: 2
t_ix_unattended: 5
t_ix_attended: 7
t_ix_unattended: 4
t_ix_attended: 2
t_ix_unattended: 9
t_ix_attended: 3
t_ix_unattended: 7
t_ix_attended: 5
t_ix_unattended: 4
t_ix_attended: 8
t_ix_unattended: 2
t_ix_attended: 4
t_ix_unattended: 5
t_ix_attended: 11
t_ix_unattended: 4
t_ix_attended: 4
t_ix_unattended: 7
t_ix_attended: 7
t_ix_unattended: 12
t_ix_attended: 2
t_ix_unattended: 5
t_ix_attended: 7
t_ix_unattended: 4
t_ix_attended: 9
t_ix_unattended: 2
t_ix_attended: 3
t_ix_unattended: 7
t_ix_attended: 4
t_ix_unattended: 5
t_ix_attended: 2
t_ix_unattended: 8
t_ix_attended: 5
t_ix_unattended: 4
t_ix_attended: 11
t_ix_unattended: 4
t_ix_attended: 4
t_ix_unattended: 7
t_ix_attended: 12
t_ix_unattended: 7
t_ix_attended: 2
t_ix_unattended: 5
t_ix_attended: 4
t_ix_unattended: 7
t_ix_attended: 9
t_ix_unattended: 2
t_ix_attended: 3
t_ix_unattended: 7
t_ix_attended: 5
t_ix_

# Approach 2: Concatenation of scores
This approach has a lower performance, so I prefer to use the previous approach.

In [116]:
def extract_epochs(X, start_idx=120, end_idx=2520, step_size=30, 
                   start_sample_offset=-24, end_sample_offset=84, 
                   amplitude_threshold=40e-6):
    """
    Function to extract epochs from time-series data for ERP features, 
    baseline-correct each epoch, and identify bad epochs based on amplitude threshold.
    
    Parameters:
    - X: Input data array of shape (n_trials, n_channels, n_samples)
    - start_idx: The starting sample index for the first epoch (default=120)
    - end_idx: The last sample index where the final epoch starts (default=2520)
    - step_size: Step size in samples, corresponding to the sliding window (default=30)
    - start_sample_offset: The offset for the start of the time window (default=-24, corresponds to -200 ms)
    - end_sample_offset: The offset for the end of the time window (default=84, corresponds to 700 ms)
    - amplitude_threshold: Threshold for identifying bad epochs based on amplitude range (default=100)
    
    Returns:
    - output_matrix: A 4D array of extracted and baseline-corrected epochs of shape 
                     (n_trials, n_epochs, n_channels, window_size)
    - bad_epochs_idx: List of indices of bad epochs for each trial and channel 
                      where amplitude range exceeds the threshold.
    """
    # Check input dimensions
    if X.ndim != 3:
        raise ValueError(f"Input X must have 3 dimensions (n_trials, n_channels, n_samples), but got {X.ndim} dimensions.")
    
    n_trials, n_channels, n_samples = X.shape
    window_size = end_sample_offset + np.abs(start_sample_offset)  # 108 samples
    epoch_timestamps = np.arange(start_idx, end_idx, step_size)    # (80,)
    n_epochs = len(epoch_timestamps)
    
    # Initialize the output matrix for the epochs and a list for bad epoch indices
    output_matrix = np.zeros((n_trials, n_epochs, n_channels, window_size))
    bad_epochs_idx = []  # To store (trial, epoch, channel) indices of bad epochs
    
    # Loop over trials, channels, and epochs to extract and baseline-correct the windows
    for i_trial in range(n_trials):
        for i_channel in range(n_channels):
            data = X[i_trial, i_channel, :]

            for i_epoch, t in enumerate(epoch_timestamps):
                epoch_start_idx = t + start_sample_offset  # Start at t - 24 samples (-200 ms)
                epoch_end_idx = t + end_sample_offset      # End at t + 84 samples (700 ms)
                
                # Ensure the window stays within bounds
                if epoch_start_idx >= 0 and epoch_end_idx <= n_samples:
                    epoch_data = data[epoch_start_idx:epoch_end_idx]
                    
                    # Baseline correction
                    baseline_mean = np.mean(epoch_data[:25])
                    epoch_data = epoch_data - baseline_mean
                    
                    # Store the epoch in the output matrix
                    output_matrix[i_trial, i_epoch, i_channel, :] = epoch_data
                    
                    # Check amplitude range after baseline subtraction
                    min_amp, max_amp = np.min(epoch_data), np.max(epoch_data)
                    amplitude_range = max_amp - min_amp
                    
                    # Log bad epochs if amplitude range exceeds threshold
                    if amplitude_range > amplitude_threshold:
                        bad_epochs_idx.append((i_trial, i_epoch, i_channel))
    
    # Return the 4D output matrix and the indices of bad epochs
    return output_matrix, bad_epochs_idx

def mark_bad_epochs(X, z, bad_idx):
    """
    Marks bad epochs in both EEG data (X) and labels (z) by setting them to NaN (or another sentinel, ie -1).

    Parameters
    ----------
    X : ndarray
        4D array of shape (n_trials, n_epochs, n_channels, n_timepoints).
    z : ndarray
        3D array of shape (n_trials, n_epochs, label_dim).
    bad_idx : list of tuples
        List of (trial, epoch, channel) indices indicating bad epochs.

    Returns
    -------
    X_marked : ndarray
        Same shape as X, with bad epochs set to NaN (or a chosen sentinel).
    z_marked : ndarray
        Same shape as z, with bad epochs set to NaN (or a chosen sentinel).
    """
    # Convert list of (trial, epoch, channel) to a set of (trial, epoch) pairs
    bad_trial_epoch_pairs = set((trial, epoch) for trial, epoch, _ in bad_idx)

    # Make copies so we don't overwrite the original arrays
    X_marked = np.copy(X)
    z_marked = np.copy(z).astype(np.float64)

    # Mark each bad epoch in both X and z
    for trial_idx, epoch_idx in bad_trial_epoch_pairs:
        X_marked[trial_idx, epoch_idx, :, :] = np.nan 
        z_marked[trial_idx, epoch_idx, :]    = np.nan 

    return X_marked, z_marked

def balance_classes(X, y, ratio_0_to_1=1.0):
    
    """
    Sub-select X and y based on a specified ratio of 0s to 1s, keeping the original order.

    Parameters:
    X (numpy.ndarray): Feature matrix of shape (n_samples, n_features).
    y (numpy.ndarray): Label vector of shape (n_samples,).
    ratio_0_to_1 (float): The desired ratio of 0s to 1s in the balanced dataset.

    Returns:
    X_balanced, y_balanced: Sub-selected feature matrix and label vector.
    """
    # Step 1: Identify indices of 0s and 1s
    indices_0 = np.where(y == 0)[0]
    indices_1 = np.where(y == 1)[0]
    
    # Step 2: Calculate the number of samples to select for each class
    num_1s = len(indices_1)
    num_0s = min(len(indices_0), int(num_1s * ratio_0_to_1))
    
    # Step 3: Randomly sample the desired number of 0s and 1s
    selected_indices_0 = np.random.choice(indices_0, num_0s, replace=False)
    selected_indices_1 = np.random.choice(indices_1, num_1s, replace=False)
    
    # Step 4: Combine selected indices and sort to preserve original order
    balanced_indices = np.sort(np.concatenate([selected_indices_0, selected_indices_1]))
    
    # Step 5: Sub-select X and y based on the balanced indices
    X_balanced = X[balanced_indices]
    y_balanced = y[balanced_indices]
    
    return X_balanced, y_balanced

def filter_valid_epochs(X, y, z=None, return_mask=False):
    """
    Filters out epochs where either the features in X or the labels in y contain NaN values.
    Optionally, if a z array is provided, it is filtered similarly.
    
    Parameters:
        X (np.ndarray): A 2D numpy array with shape (n_epochs, n_features).
        y (np.ndarray): A 1D numpy array with shape (n_epochs,).
        z (np.ndarray, optional): An array that will be filtered using the same mask.
        return_mask (bool, optional): If True, the boolean mask used for filtering is returned.
    
    Returns:
        filtered_X (np.ndarray): X with only rows that have no NaN values.
        filtered_y (np.ndarray): y with only entries corresponding to valid epochs.
        filtered_z (np.ndarray or None): Filtered z array (if provided) or None.
        mask (np.ndarray, optional): The boolean mask of valid epochs; only returned if return_mask=True.
    """
    # Create a mask for valid labels and features
    valid_label_mask = ~np.isnan(y)
    valid_feature_mask = ~np.isnan(X).any(axis=1)
    combined_mask = valid_label_mask & valid_feature_mask

    # Apply the mask to X and y
    filtered_X = X[combined_mask]
    filtered_y = y[combined_mask]
    
    if z is not None:
        filtered_z = z[combined_mask]
    else:
        filtered_z = None

    if return_mask:
        return filtered_X, filtered_y, filtered_z, combined_mask
    else:
        return filtered_X, filtered_y, filtered_z
    
def extract_features_from_X(X_matrix, ToI = None):
    """
    Extracts the maximum amplitudes from specified time ranges for each trial, epoch, and channel in the input data.

    Parameters:
    - X_matrix: A 4D numpy array of shape (n_trials, n_epochs, n_channels, n_samples) representing the input data.
    - ToI: A list of tuples, where each tuple contains the start and end indices of a time range of interest.

    Returns:
    - feature_matrix: A 4D numpy array of shape (n_trials, n_epochs, n_channels, len(ToI)) containing the maximum
                      values from the specified time ranges for each trial, epoch, and channel.
    """
    # Extract the shape of the input matrix
    n_trials, n_epochs, n_channels, n_samples = X_matrix.shape 
    
    # Initialize the feature matrix to store maximum values for each time range
    feature_matrix = np.zeros((n_trials, n_epochs, n_channels, len(ToI)))

    # Loop over the time ranges (ToI) and extract the max value for each range
    for i_range, (start, end) in enumerate(ToI):
        # For each time range, find the maximum values along the last axis (time samples) in the specified range
        feature_matrix[ :, :, :, i_range] = np.mean((X_matrix[ :, :, :, start:end]), axis=-1)

    # Return the feature matrix
    return feature_matrix

# Function to apply baseline correction
def apply_baseline_correction(epochs, baseline_start_sample, baseline_end_sample):
    baseline = np.mean(epochs[:, :, baseline_start_sample:baseline_end_sample], axis=2, keepdims=True)
    return epochs - baseline  # Subtract baseline from all samples in the epoch

In [58]:
import numpy as np
import os
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from mne.time_frequency import psd_array_multitaper
import warnings
import mne
from sklearn.covariance import LedoitWolf
from scipy.signal import welch
from sklearn.metrics import precision_recall_curve, auc
from scipy.stats import pearsonr



# Suppress all warnings
mne.set_log_level('warning')

# Directory containing the preprocessed data
file_dir = '/Users/juliette/Desktop/thesis/preprocessing/hybrid_preprocessing'
decoding_results_dir = '/Users/juliette/Desktop/thesis/results/alpha+p300'

# Define the alpha range for PSD calculation
min_bin = 8
max_bin = 12
rejection_threshold = 60e-6
discard_threshold = 20


# Initialize results storage
results = []
fold_pr_auc = []
fold_correct_trials = []

# List of subjects
subjects = ["VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij",
            "VPpdik", "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit",
            "VPpdiu", "VPpdiv", "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"]

subjects = ["VPpdik", "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit"]
# Load Data for each subject
for subject in subjects:
    print(f"Processing {subject}...")
    # Load preprocessed data
    fn = os.path.join(file_dir, f"sub-{subject}_task-covert_c-VEP+P300_ICA.npz")
    tmp = np.load(fn)

    X = tmp["X"]  # EEG data matrix (trials, channels, samples)
    y = tmp["y"]  # Labels
    z = tmp["z"]  # Target presence (trials, epochs, sides)
    V = tmp["V"]  # One code cycle (classes, samples)
    fs = tmp["fs"].flatten()[0]
    nyquist_freq = fs // 2

    # Cross-validation
    fold_accuracies = []
    n_folds = 4
    n_trials = X.shape[0] // n_folds
    folds = np.repeat(np.arange(n_folds), n_trials)

    new_feature_vectors = []

    for i_fold in range(n_folds):
        print(f"  Fold {i_fold + 1}/{n_folds}")

        # Split train and test data
        X_trn, y_trn, z_trn = X[folds != i_fold, :, :], y[folds != i_fold], z[folds != i_fold, :, :]
        X_tst, y_tst, z_tst = X[folds == i_fold, :, :], y[folds == i_fold], z[folds == i_fold, :, :]

        
        # --- P300 ---
        # - Obtaining target-locked epochs - 
        
        # Extract epochs
        epochs_trn, bad_epochs_trn = extract_epochs(X_trn, amplitude_threshold=rejection_threshold)
        epochs_tst, bad_epochs_tst = extract_epochs(X_tst, amplitude_threshold=rejection_threshold)
        print("Shape of epochs_trn:", epochs_trn.shape)
        print("Shape of epochs_tst:", epochs_tst.shape)

        # Mark bad epochs in both EEG data and labels
        X_clean_trn, z_clean_trn = mark_bad_epochs(epochs_trn, z_trn, bad_epochs_trn)
        X_clean_tst, z_clean_tst = mark_bad_epochs(epochs_tst, z_tst, bad_epochs_tst)
        print("Shape of X_clean_trn:", X_clean_trn.shape)
        print("Shape of X_clean_tst:", X_clean_tst.shape)
        
        # Extract features from valid epochs. Shape: (trials, epochs, channels, samples)
        ToI = [(30, 38), (38, 48), (48, 57), (57, 69), (69, 87), (87, 108)]
        features_trn = extract_features_from_X(X_clean_trn, ToI)
        features_tst = extract_features_from_X(X_clean_tst, ToI)
        print("Shape of features_trn:", features_trn.shape)
        print("Shape of features_tst:", features_tst.shape)
        
        # - Training -
        # Flatten training trials into epochs: shape becomes (n_trials * epochs, channel * features)
        features_trn = features_trn.reshape(-1, features_trn.shape[2] * features_trn.shape[3])
        
        # Extract labels for training epochs using z and y 
        trial_indices_trn = np.arange(len(y_trn))  # indices for trials
        y_trn_epochs = z_trn[trial_indices_trn, :, y_trn].reshape(-1)
        
        X_trn_epochs = np.nan_to_num(features_trn, nan=0)
        y_trn_epochs = np.nan_to_num(y_trn_epochs, nan=0)
        
        print("Shape of X_trn_epochs (this is fed into LDA):", X_trn_epochs.shape)
        print("Shape of y_trn_epochs (this is fed into LDA):", y_trn_epochs.shape)
        
        # Fit LDA
        lda = LDA(solver="lsqr", covariance_estimator=LedoitWolf())
        lda.fit(X_trn_epochs, y_trn_epochs)  # Dimensionality bust be 2

        # - Testing -
        # Flatten testing trials into epochs
        X_tst_epochs = features_tst.reshape(-1, features_tst.shape[2] * features_tst.shape[3])
        
        # Extract labels for testing epochs (again, using z and y for indexing)
        trial_indices_tst = np.arange(len(y_tst))
        y_tst_epochs = z_tst[trial_indices_tst, :, y_tst].reshape(-1)
        
        # Reshape z_tst_trials into epochs.
        z_tst_epochs = z_tst.reshape(len(y_tst) * z_tst.shape[1], 2)
        
        # Filter testing epochs and also retrieve the original mask so we can count epochs per trial
        X_tst_epochs = np.nan_to_num(X_tst_epochs, nan=0)
        y_tst_epochs = np.nan_to_num(y_tst_epochs, nan=0)
        
        # Filter testing epochs and also retrieve the original mask so we can count epochs per trial
        X_tst_epochs, y_tst_epochs, z_tst_epochs, combined_mask_tst = filter_valid_epochs(
            X_tst_epochs, y_tst_epochs, z=z_tst_epochs, return_mask=True
        )
        
        # Calculate the number of preserved epochs per trial for testing.
        # Here, combined_mask_tst still has the original shape before filtering.
        # Reshape it to [n_trials, epochs_per_trial] and sum True values per trial.
        epoch_counts = combined_mask_tst.reshape(len(y_tst), -1)
        num_epochs = np.sum(epoch_counts, axis=1)
        
        # Rebuild trial structure for testing data based on num_epochs
        nested_X_tst_trials = []
        nested_z_tst_epochs = []
        start_idx = 0
        for trial_idx, n_ep in enumerate(num_epochs):
            end_idx = start_idx + n_ep
            nested_X_tst_trials.append(X_tst_epochs[start_idx:end_idx])
            nested_z_tst_epochs.append(z_tst_epochs[start_idx:end_idx])
            start_idx = end_idx
        

        # Evaluate model on test data
        correct_trials = 0
        
        for t_idx in range(len(y_tst)):

            num_preserved_epochs = num_epochs[t_idx]
            if num_preserved_epochs < discard_threshold:
                discarded_trial_counter +=1
                continue
            
            # Log cued side informed by y_tst
            cued_side = y_tst[t_idx]
            # create event vectors & ground truth
            left_targets = nested_z_tst_epochs[t_idx][:, 0]
            right_targets = nested_z_tst_epochs[t_idx][:, 1]
            cued_targets = nested_z_tst_epochs[t_idx] [:, cued_side]
   
            # Compute LDA scores for epochs
            epoch_scores = lda.decision_function(nested_X_tst_trials[t_idx])
            
            # Log performance per fold
            precision, recall, _ = precision_recall_curve(cued_targets, epoch_scores)
            pr_auc_score = auc(recall, precision)
            fold_pr_auc.append(pr_auc_score)

            # Correlation-based decision
            corr_left, _ = pearsonr(epoch_scores, left_targets)
            corr_right, _ = pearsonr(epoch_scores, right_targets)

            # Trial-level decision rule based on correlation
            decision = 0 if corr_left > corr_right else 1
            if decision == cued_side:
                correct_trials += 1


        # --- Alpha Extraction ---

        # Extract alpha features. For all trials, average over frequency bin (8-12 Hz) per channel using Welch's method
        psd_features_trn = np.array([
            welch(trial, fs=fs, nperseg=nyquist_freq, scaling='density')[1] # Set number of data points in each segment to the Nyquist frequency
            [:, (min_bin <= freqs) & (freqs <= max_bin)].mean(axis=1) # Selects only the frequencies between min_bin and max_bin and averages over all channels
            for trial, freqs in [(X_trn[i], welch(X_trn[i][0], fs=fs, nperseg=fs//2)[0]) 
            for i in range(X_trn.shape[0])] # For each trial, pair its EEG data with frequency bins computed from the first channel's Welch PSD) to prepare for bandpower analysis
        ])
        
        psd_features_tst = np.array([
            welch(trial, fs=fs, nperseg=nyquist_freq, scaling='density')[1] # Set number of data points in each segment to the Nyquist frequency
            [:, (min_bin <= freqs) & (freqs <= max_bin)].mean(axis=1) # Selects only the frequencies between min_bin and max_bin and averages over all channels
            for trial, freqs in [(X_tst[i], welch(X_tst[i][0], fs=fs, nperseg=fs//2)[0])
            for i in range(X_tst.shape[0])] # For each trial, pair its EEG data with frequency bins computed from the first channel's Welch PSD) to prepare for bandpower analysis
        ])


        # --- Concatenation ---
        # Concatenate the P300 correlation and the alpha PSD features
        combined_features_trn = np.column_stack([corr_left * np.ones(X_trn.shape[0]),  # Left stimulus correlation
                                                 corr_right * np.ones(X_trn.shape[0]),  # Right stimulus correlation
                                                 psd_features_trn])  # Alpha PSD features

        combined_features_tst = np.column_stack([corr_left * np.ones(X_tst.shape[0]),  # Left stimulus correlation
                                                 corr_right * np.ones(X_tst.shape[0]),  # Right stimulus correlation
                                                 psd_features_tst])  # Alpha PSD features

        # Replace NaN values with 0
        combined_features_trn = np.nan_to_num(combined_features_trn, nan=0)
        combined_features_tst = np.nan_to_num(combined_features_tst, nan=0)

        print(f"Shape of combined_features_trn: {combined_features_trn.shape}")
        print(f"Shape of combined_features_tst: {combined_features_tst.shape}")

        # --- Ensemble ---
        # Train final LDA on new feature vectors (combined features)
        lda_combine = LDA(solver="lsqr", covariance_estimator=LedoitWolf())
        lda_combine.fit(combined_features_trn, y_trn)

        # Final prediction
        y_final = lda_combine.predict(combined_features_tst)

        # Calculate accuracy
        accuracy = np.mean(y_final == y_tst)
        fold_accuracies.append(accuracy)

        print(f"Fold {i_fold + 1} Accuracy: {accuracy:.3f}")

    # Compute subject-level results
    accuracy = np.round(np.mean(fold_accuracies), 2)
    se = np.round(np.std(fold_accuracies) / np.sqrt(n_folds), 2)
    results.append((subject, accuracy, se))

    # Print average accuracy per subject
    print(f"Average Accuracy for {subject}: {accuracy:.3f}")

# # Save results
# if not os.path.exists(decoding_results_dir):
#         os.makedirs(decoding_results_dir)
# results_save_path = join(decoding_results_dir, f"covert_alpha_p300_results_concat.npy")     
# np.save(results_save_path, results_array)    

# Convert results to a structured numpy array
results_array = np.array(
    results, dtype=[('subject', 'U10'), ('accuracy', 'f4'), ('standard_error', 'f4')]
)

# Overall results
overall_accuracy = np.round(results_array['accuracy'].mean(), 2)
overall_se = np.round(results_array['standard_error'].mean(), 2)
print(f"Overall LDA accuracy with PSD: {overall_accuracy:.2f} ± {overall_se:.2f}")

Processing VPpdik...
  Fold 1/4
Shape of epochs_trn: (60, 80, 63, 108)
Shape of epochs_tst: (20, 80, 63, 108)
Shape of X_clean_trn: (60, 80, 63, 108)
Shape of X_clean_tst: (20, 80, 63, 108)
Shape of features_trn: (60, 80, 63, 6)
Shape of features_tst: (20, 80, 63, 6)
Shape of X_trn_epochs (this is fed into LDA): (4800, 378)
Shape of y_trn_epochs (this is fed into LDA): (4800,)
Shape of combined_features_trn: (60, 65)
Shape of combined_features_tst: (20, 65)
Fold 1 Accuracy: 0.500
  Fold 2/4
Shape of epochs_trn: (60, 80, 63, 108)
Shape of epochs_tst: (20, 80, 63, 108)
Shape of X_clean_trn: (60, 80, 63, 108)
Shape of X_clean_tst: (20, 80, 63, 108)
Shape of features_trn: (60, 80, 63, 6)
Shape of features_tst: (20, 80, 63, 6)
Shape of X_trn_epochs (this is fed into LDA): (4800, 378)
Shape of y_trn_epochs (this is fed into LDA): (4800,)
Shape of combined_features_trn: (60, 65)
Shape of combined_features_tst: (20, 65)
Fold 2 Accuracy: 0.650
  Fold 3/4
Shape of epochs_trn: (60, 80, 63, 108)
S

/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:172: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_left, _ = pearsonr(epoch_scores, left_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_right, _ = pearsonr(epoch_scores, right_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:172: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_left, _ = pearsonr(epoch_scores, left_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_right, _ = pearsonr(epoch_scores, right_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/288

Shape of combined_features_trn: (60, 66)
Shape of combined_features_tst: (20, 66)
Fold 2 Accuracy: 0.500
  Fold 3/4
Shape of epochs_trn: (60, 80, 64, 108)
Shape of epochs_tst: (20, 80, 64, 108)
Shape of X_clean_trn: (60, 80, 64, 108)
Shape of X_clean_tst: (20, 80, 64, 108)
Shape of features_trn: (60, 80, 64, 6)
Shape of features_tst: (20, 80, 64, 6)
Shape of X_trn_epochs (this is fed into LDA): (4800, 384)
Shape of y_trn_epochs (this is fed into LDA): (4800,)


/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:172: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_left, _ = pearsonr(epoch_scores, left_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_right, _ = pearsonr(epoch_scores, right_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:172: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_left, _ = pearsonr(epoch_scores, left_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_right, _ = pearsonr(epoch_scores, right_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/288

Shape of combined_features_trn: (60, 66)
Shape of combined_features_tst: (20, 66)
Fold 3 Accuracy: 0.950
  Fold 4/4
Shape of epochs_trn: (60, 80, 64, 108)
Shape of epochs_tst: (20, 80, 64, 108)
Shape of X_clean_trn: (60, 80, 64, 108)
Shape of X_clean_tst: (20, 80, 64, 108)
Shape of features_trn: (60, 80, 64, 6)
Shape of features_tst: (20, 80, 64, 6)
Shape of X_trn_epochs (this is fed into LDA): (4800, 384)
Shape of y_trn_epochs (this is fed into LDA): (4800,)


...

/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:172: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_left, _ = pearsonr(epoch_scores, left_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_right, _ = pearsonr(epoch_scores, right_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:172: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_left, _ = pearsonr(epoch_scores, left_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_right, _ = pearsonr(epoch_scores, right_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/288

Shape of combined_features_trn: (60, 66)
Shape of combined_features_tst: (20, 66)
Fold 3 Accuracy: 0.800
  Fold 4/4
Shape of epochs_trn: (60, 80, 64, 108)
Shape of epochs_tst: (20, 80, 64, 108)
Shape of X_clean_trn: (60, 80, 64, 108)
Shape of X_clean_tst: (20, 80, 64, 108)
Shape of features_trn: (60, 80, 64, 6)
Shape of features_tst: (20, 80, 64, 6)
Shape of X_trn_epochs (this is fed into LDA): (4800, 384)
Shape of y_trn_epochs (this is fed into LDA): (4800,)


/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:172: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_left, _ = pearsonr(epoch_scores, left_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_right, _ = pearsonr(epoch_scores, right_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:172: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_left, _ = pearsonr(epoch_scores, left_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/2882666468.py:173: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_right, _ = pearsonr(epoch_scores, right_targets)
/var/folders/qx/1c8hj9_92533cb1kgn4q_yt80000gn/T/ipykernel_23621/288

Shape of combined_features_trn: (60, 66)
Shape of combined_features_tst: (20, 66)
Fold 4 Accuracy: 0.750
Average Accuracy for VPpdio: 0.860
Processing VPpdip...
  Fold 1/4
Shape of epochs_trn: (60, 80, 64, 108)
Shape of epochs_tst: (20, 80, 64, 108)
Shape of X_clean_trn: (60, 80, 64, 108)
Shape of X_clean_tst: (20, 80, 64, 108)
Shape of features_trn: (60, 80, 64, 6)
Shape of features_tst: (20, 80, 64, 6)
Shape of X_trn_epochs (this is fed into LDA): (4800, 384)
Shape of y_trn_epochs (this is fed into LDA): (4800,)
Shape of combined_features_trn: (60, 66)
Shape of combined_features_tst: (20, 66)
Fold 1 Accuracy: 0.500
  Fold 2/4
Shape of epochs_trn: (60, 80, 64, 108)
Shape of epochs_tst: (20, 80, 64, 108)
Shape of X_clean_trn: (60, 80, 64, 108)
Shape of X_clean_tst: (20, 80, 64, 108)
Shape of features_trn: (60, 80, 64, 6)
Shape of features_tst: (20, 80, 64, 6)
Shape of X_trn_epochs (this is fed into LDA): (4800, 384)
Shape of y_trn_epochs (this is fed into LDA): (4800,)
Shape of combine

KeyboardInterrupt: 

In [177]:

import os
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from mne.time_frequency import psd_array_welch
import mne

# Suppress warnings from MNE
mne.set_log_level('warning')

# Define path
file_dir = '/Users/juliette/Desktop/thesis/preprocessing/hybrid_preprocessing'
decoding_results_dir = '/Users/juliette/Desktop/thesis/results/alpha+p300'

# Set parameters
window_size = 2     # in seconds for alpha-band sliding window
step_size = 0.5     # sliding step
min_bin, max_bin = 8, 13  # alpha band

# Subjects
subjects = ["VPpdia", "VPpdib", "VPpdic", "VPpdid", "VPpdie", "VPpdif", "VPpdig", "VPpdih", "VPpdii", "VPpdij",
            "VPpdik", "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit",
            "VPpdiu", "VPpdiv", "VPpdiw", "VPpdix", "VPpdiy", "VPpdiz", "VPpdiza", "VPpdizb", "VPpdizc"]

subjects = ["VPpdik", "VPpdil", "VPpdim", "VPpdin", "VPpdio", "VPpdip", "VPpdiq", "VPpdir", "VPpdis", "VPpdit"]
# Initialize results storage
results = []

# Loop over the subjects
for subject in subjects:
    print(f"Processing {subject}...")

    # Load preprocessed ICA-cleaned data
    fn = os.path.join(file_dir, f"sub-{subject}_task-covert_c-VEP+P300_ICA.npz")
    tmp = np.load(fn, allow_pickle=True)

    X = tmp["X"]  # shape: trials × channels × samples
    y = tmp["y"]  # binary trial label (e.g., attention)
    z = tmp["z"]  # stimulus labels
    V = tmp["V"]  # marker stream, used for timing info
    fs = int(tmp["fs"].flatten()[0])
    n_trials, n_channels, n_samples = X.shape

    # Split folds
    fold_accuracies = []
    n_folds = 4
    n_per_fold = n_trials // n_folds
    folds = np.repeat(np.arange(n_folds), n_per_fold)

    for i_fold in range(n_folds):
        print(f"  Fold {i_fold + 1}/{n_folds}")

        # Train/test split
        X_trn, y_trn, z_trn = X[folds != i_fold], y[folds != i_fold], z[folds != i_fold]
        X_tst, y_tst, z_tst = X[folds == i_fold], y[folds == i_fold], z[folds == i_fold]

        # Frequency analysis for the alpha band
        def extract_alpha(X_trials):
            alpha_feats = []
            for trial in X_trials:
                trial_alpha = [] # This variable will hold the alpha power values for each time window in the current trial
                total_duration = n_samples/fs
        
                for start in np.arange(0, total_duration - window_size, step_size):
                    start_ix = int(start * fs) # convert the starting times to sample indices
                    stop_ix = int(start_ix + window_size * fs)
                    
                    # If stop_ix exceeds the time samples in the trial the window is too large and it stops
                    if stop_ix > trial.shape[1]:
                        break
                    
                    # This contains the EEG data for the selected time window
                    # It is sliced from the trial array for all channels, and the selected time window
                    segment = trial[:, start_ix:stop_ix]
                    
                    # Adjust n_fft and use n_per_seg to avoid the error
                    psds, freqs = psd_array_welch(
                        segment, sfreq=fs, fmin=min_bin, fmax=max_bin, n_per_seg=segment.shape[1] # The number of points per segment 
                    )                                                                             # is equal to the length of the segment
                    alpha_power = psds.mean(axis=1)  # average power across the frequency bins in the alpha band across channels
                    trial_alpha.append(alpha_power)
                trial_alpha = np.mean(trial_alpha, axis=0)  # calculate single alpha power feature for the trial
                alpha_feats.append(trial_alpha)
            return np.array(alpha_feats)  # shape: (trials × channels)

        alpha_trn = extract_alpha(X_trn)
        alpha_tst = extract_alpha(X_tst)


        # --- STEP 4: ERP FEATURE EXTRACTION FOR P300 ---
        def extract_erp_no_bins(X_trials, z_trials, y, fs, n_channels, amplitude_threshold=40e-6, start_sample_offset=-24, end_sample_offset=84, step_size=30):
            # An empty list that will hold the ERP features for each trial
            erp_feats = []

            # Loop through each trial, stimulus coding, and attended side label
            for trial, z, target_side in zip(X_trials, z_trials, y):
                cued_side = target_side  # Assuming 'y' contains the trial information about which side (left or right)

                # Create event vectors for left and right targets
                left_targets = z[:, 0]  # Assuming z contains the event markers, where 0 = left, 1 = right
                right_targets = z[:, 1]
                attended_stimuli = z[:, cued_side]  # Select the cued target side

                # Identify left and right target events
                z_left = left_targets == 1
                z_right = right_targets == 1

                # Determine attended and unattended sides
                if target_side == 0:  # Attending to left
                    attended_side = z_left  # Left target is attended
                    unattended_side = z_right  # Right target is unattended
                else:  # Attending to right
                    attended_side = z_right  # Right target is attended
                    unattended_side = z_left  # Left target is unattended

                # Find the indices of targets on the attended side
                target_indices_attended = np.where(attended_side == 1)[0]
                # Find the indices of targets on the unattended side
                target_indices_unattended = np.where(unattended_side == 1)[0]

                # If no target indices are found for both the attended and unattended sides, append a placeholder
                if len(target_indices_attended) == 0 and len(target_indices_unattended) == 0:
                    erp_feats.append(np.zeros(n_channels * 2))  # Placeholder (attended + unattended)
                    continue

                # Initialize arrays that will hold the ERP features for the attended and unattended targets
                feat_attended = np.zeros(n_channels)
                feat_unattended = np.zeros(n_channels)

                # Process attended target
                if len(target_indices_attended) > 0:
                    # Extract the first target index, which gives the position of the attended target
                    t_ix_attended = target_indices_attended[0]

                    # Extract epochs around the attended target
                    epoch_attended, _ = extract_epochs(trial[None, :, :], start_idx=120, end_idx=2520, step_size=30, 
                           start_sample_offset=-24, end_sample_offset=84, 
                           amplitude_threshold=40e-6)

                    # Compute the ERP feature for the attended target by averaging the EEG data across the time axis
                    feat_attended = epoch_attended.mean(axis=(1, 2))  # Averaging across epochs and channels

                # Process unattended target
                if len(target_indices_unattended) > 0:
                    # Extract the first target index, which gives the position of the unattended target
                    t_ix_unattended = target_indices_unattended[0]

                    # Extract epochs around the unattended target
                    epoch_unattended, _ = extract_epochs(trial[None, :, :], start_idx=120, end_idx=2520, step_size=30, 
                           start_sample_offset=-24, end_sample_offset=84, 
                           amplitude_threshold=40e-6)

                    # Compute the ERP feature for the unattended target by averaging the EEG data across the time axis
                    feat_unattended = epoch_unattended.mean(axis=(1, 2))  # Averaging across epochs and channels

                # Concatenate attended and unattended features and append to the list
                erp_feats.append(np.concatenate([feat_attended, feat_unattended]))

            return np.array(erp_feats)


        # Extract ERP features for training and testing
        erp_trn = extract_erp_no_bins(X_trn, z_trn, y_trn, fs, n_channels)
        erp_tst = extract_erp_no_bins(X_tst, z_tst, y_tst, fs, n_channels)
        
        # Flatten erp_trn and erp_tst to be 2D (trials × features)
        erp_trn = erp_trn.reshape(erp_trn.shape[0], -1)  # Flatten to (trials × features)
        erp_tst = erp_tst.reshape(erp_tst.shape[0], -1)  # Flatten to (trials × features)

        # Now you can concatenate alpha_trn with erp_trn, and alpha_tst with erp_tst
        F_trn = np.concatenate([alpha_trn, erp_trn], axis=1)
        F_tst = np.concatenate([alpha_tst, erp_tst], axis=1)

        # Handle NaNs, replaces any NaN values with 0
        F_trn[np.isnan(F_trn)] = 0
        F_tst[np.isnan(F_tst)] = 0

        # Normalize, it scales the features so that they have a mean of 0 and a standard deviation of 1
        scaler = StandardScaler()
        F_trn = scaler.fit_transform(F_trn)
        F_tst = scaler.transform(F_tst)

        # --- STEP 6: CLASSIFICATION ---
        clf = LDA(solver='lsqr', shrinkage='auto')  # Ledoit-Wolf shrinkage
        clf.fit(F_trn, y_trn)
        y_pred = clf.predict(F_tst)

        acc = np.mean(y_pred == y_tst)
        fold_accuracies.append(acc)
        print(f"    Fold accuracy: {acc:.2f}")

    # --- STEP 7: SAVE RESULTS FOR THIS SUBJECT ---
    acc_mean = np.round(np.mean(fold_accuracies), 2)
    acc_se = np.round(np.std(fold_accuracies) / np.sqrt(n_folds), 2)
    results.append((subject, acc_mean, acc_se))
    print(f"{subject}: Accuracy = {acc_mean:.2f}, SE = {acc_se:.2f}")

    
# --- STEP 8: AVERAGE ACCURACY OVER ALL SUBJECTS ---
all_accuracies = [result[1] for result in results]
mean_accuracy_all = np.mean(all_accuracies)
se_accuracy_all = np.std(all_accuracies) / np.sqrt(len(all_accuracies))

print(f"\nAverage Accuracy across all subjects: {mean_accuracy_all:.2f}")
print(f"Standard Error of Accuracy: {se_accuracy_all:.2f}")

Processing VPpdik...
  Fold 1/4
    Fold accuracy: 0.50
  Fold 2/4
    Fold accuracy: 0.45
  Fold 3/4
    Fold accuracy: 0.60
  Fold 4/4
    Fold accuracy: 0.70
VPpdik: Accuracy = 0.56, SE = 0.05
Processing VPpdil...
  Fold 1/4
    Fold accuracy: 0.85
  Fold 2/4
    Fold accuracy: 0.95
  Fold 3/4
    Fold accuracy: 1.00
  Fold 4/4
    Fold accuracy: 1.00
VPpdil: Accuracy = 0.95, SE = 0.03
Processing VPpdim...
  Fold 1/4
    Fold accuracy: 0.85
  Fold 2/4
    Fold accuracy: 0.90
  Fold 3/4
    Fold accuracy: 0.95
  Fold 4/4
    Fold accuracy: 0.95
VPpdim: Accuracy = 0.91, SE = 0.02
Processing VPpdin...
  Fold 1/4
    Fold accuracy: 0.60
  Fold 2/4
    Fold accuracy: 0.70
  Fold 3/4
    Fold accuracy: 0.80
  Fold 4/4
    Fold accuracy: 0.75
VPpdin: Accuracy = 0.71, SE = 0.04
Processing VPpdio...
  Fold 1/4
    Fold accuracy: 0.95
  Fold 2/4
    Fold accuracy: 0.90
  Fold 3/4
    Fold accuracy: 0.85
  Fold 4/4
    Fold accuracy: 0.90
VPpdio: Accuracy = 0.90, SE = 0.02
Processing VPpdip...

In [144]:
print(X_attended)

NameError: name 'X_attended' is not defined